In [79]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
import utils
from environment import Environment
from agent import Agent

In [ ]:
N = 4
max_episodes = 700
max_steps = 7000
learning_rate = 1.0
epsilon_start = 0.8
epsilon_end = 0.6
default_state_value = 0.0
epsilon_decay_type = "linear"   # Should be "linear" or "exponential"
update_on_increase = True

# env = Environment(N)

# agent = Agent(env)

# initial_state_string = utils.get_random_state_string(N)

agent.async_value_iteration_with_stack(initial_state_string, 
                                       max_episodes, 
                                       max_steps, 
                                       learning_rate, 
                                       epsilon_start, 
                                       epsilon_end, 
                                       default_state_value, 
                                       epsilon_decay_type, 
                                       update_on_increase)

In [ ]:
N = 3
max_episodes = 3000
max_steps = 2000
learning_rate = 1.0
epsilon_start = 0.1
epsilon_end = 0.1
default_state_value = 0.0
theta = 1e-3

env = Environment(N)

agent = Agent(env)

initial_state_string = utils.get_random_state_string(N)

agent.async_value_iteration(initial_state_string, max_episodes, max_steps, learning_rate, epsilon_start, epsilon_end, default_state_value, theta)

In [132]:
N = 5
n = 7000
max_episodes = 2000
max_steps = 7000
epsilon_start = 0.85
epsilon_end = 0.55
default_state_value = 0.0
update_on_increase = True
epsilon_decay_type = "linear"
plus_value_iteration = True
plus_value_iteration_with_stack = True


# env = Environment(N)

# agent = Agent(env)

# initial_state_string = utils.get_random_state_string(N)

agent.n_step_TD(initial_state_string, 
                n, 
                max_episodes, 
                max_steps, 
                epsilon_start, 
                epsilon_end, 
                default_state_value, 
                update_on_increase, 
                epsilon_decay_type, 
                plus_value_iteration, 
                plus_value_iteration_with_stack)

------ Episode 799 successful ------
------ Episode 803 successful ------
------ Episode 832 successful ------
------ Episode 834 successful ------
------ Episode 838 successful ------
------ Episode 843 successful ------
------ Episode 846 successful ------
------ Episode 849 successful ------
------ Episode 853 successful ------
------ Episode 855 successful ------
------ Episode 858 successful ------
------ Episode 860 successful ------
------ Episode 864 successful ------
------ Episode 865 successful ------
------ Episode 867 successful ------
------ Episode 868 successful ------
------ Episode 869 successful ------
------ Episode 870 successful ------
------ Episode 871 successful ------
------ Episode 873 successful ------
------ Episode 875 successful ------
------ Episode 876 successful ------
------ Episode 877 successful ------
------ Episode 879 successful ------
------ Episode 880 successful ------
------ Episode 881 successful ------
------ Episode 882 successful ------
-

In [ ]:
N = 3
n = 50
max_episodes = 300
max_steps = 1000
epsilon_start = 0.5
epsilon_end = 0.01
default_state_value = 0.0


env = Environment(N)

agent = Agent(env)

initial_state_string = utils.get_random_state_string(N)

agent.n_step_TD_2(initial_state_string, n, max_episodes, max_steps, epsilon_start, epsilon_end, default_state_value)

In [140]:
len(agent.V)

7969624

In [134]:
agent.V[initial_state_string]

98513.77083333369

In [139]:
agent.exploit(initial_state_string)

7  1  17 16 12 
10 13 3  22 11 
0  2  23 4  8  
21 9  24 19 5  
15 18 20 6  14 
- step: 1 -
7  1  17 16 12 
10 13 3  22 11 
2  0  23 4  8  
21 9  24 19 5  
15 18 20 6  14 
- step: 2 -
7  1  17 16 12 
10 0  3  22 11 
2  13 23 4  8  
21 9  24 19 5  
15 18 20 6  14 
- step: 3 -
7  1  17 16 12 
0  10 3  22 11 
2  13 23 4  8  
21 9  24 19 5  
15 18 20 6  14 
- step: 4 -
0  1  17 16 12 
7  10 3  22 11 
2  13 23 4  8  
21 9  24 19 5  
15 18 20 6  14 
- step: 5 -
1  0  17 16 12 
7  10 3  22 11 
2  13 23 4  8  
21 9  24 19 5  
15 18 20 6  14 
- step: 6 -
1  17 0  16 12 
7  10 3  22 11 
2  13 23 4  8  
21 9  24 19 5  
15 18 20 6  14 
- step: 7 -
1  17 3  16 12 
7  10 0  22 11 
2  13 23 4  8  
21 9  24 19 5  
15 18 20 6  14 
- step: 8 -
1  17 3  16 12 
7  0  10 22 11 
2  13 23 4  8  
21 9  24 19 5  
15 18 20 6  14 
- step: 9 -
1  0  3  16 12 
7  17 10 22 11 
2  13 23 4  8  
21 9  24 19 5  
15 18 20 6  14 
- step: 10 -
1  3  0  16 12 
7  17 10 22 11 
2  13 23 4  8  
21 9  24 19 5  
15 18 20 6  14 